# チャットモデルで関数を呼び出す方法
このノートでは、GPTモデルの機能を拡張するために、チャット補完APIを外部関数と組み合わせて使用する方法について説明します。

functionsはチャット補完APIのオプションパラメータで、関数の仕様を提供するために使用することができます。この目的は、モデルが提供された仕様に準拠した関数引数を生成することを可能にすることです。なお、APIは実際に関数の呼び出しを実行することはありません。モデルの出力を使って関数呼び出しを実行するのは、開発者次第です。

functionsパラメータが提供されている場合、デフォルトでは、モデルは関数のいずれかを使用することが適切である場合に決定します。APIは、function_callパラメータを{"name"}に設定することで、特定の関数を使用するように強制することができます： "<insert-function-name>"}とすることで、特定の関数の使用を強制することができる。また、function_callパラメータを "none "に設定することで、どの関数も使用しないように強制することも可能である。関数が使用された場合、出力には "finish_reason "が含まれます： 「また、関数の名前と生成された関数の引数を持つfunction_callオブジェクトが出力されます。

概要
このノートには、次の2つのセクションがあります：

関数の引数を生成する方法： 関数の引数を生成する方法：関数のセットを指定し、APIを使用して関数の引数を生成します。
モデル生成された引数で関数を呼び出す方法： モデル生成された引数を持つ関数を実際に実行することで、ループを閉じます。

www.DeepL.com/Translator（無料版）で翻訳しました。

## 関数の引数を生成する方法

In [3]:
%pip install scipy
%pip install tenacity
%pip install tiktoken
%pip install termcolor
%pip install openai
%pip install requests

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached tenacity-8.2.2-py3-none-any.whl (24 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



  Using cached tiktoken-0.4.0-cp310-cp310-win_amd64.whl (635 kB)
  Using cached requests-2.31.0-py3-none-any.whl (62 kB)
  Using cached regex-2023.6.3-cp310-cp310-win_amd64.whl (268 kB)
  Using cached urllib3-2.0.3-py3-none-any.whl (123 kB)
  Using cached charset_normalizer-3.1.0-cp310-cp310-win_amd64.whl (97 kB)
  Using cached idna-3.4-py3-none-any.whl (61 kB)
  Using cached certifi-2023.5.7-py3-none-any.whl (156 kB)


...


[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached openai-0.27.8-py3-none-any.whl (73 kB)
  Using cached tqdm-4.65.0-py3-none-any.whl (77 kB)
  Using cached aiohttp-3.8.4-cp310-cp310-win_amd64.whl (319 kB)
  Using cached attrs-23.1.0-py3-none-any.whl (61 kB)
  Using cached yarl-1.9.2-cp310-cp310-win_amd64.whl (61 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl (7.6 kB)
  Using cached multidict-6.0.4-cp310-cp310-win_amd64.whl (28 kB)
  Using cached async_timeout-4.0.2-py3-none-any.whl (5.8 kB)
  Using cached frozenlist-1.3.3-cp310-cp310-win_amd64.whl (33 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [21]:
import json
import openai
import requests
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored

GPT_MODEL = "gpt-4-0613"

## ユーティリティ
まず、Chat Completions APIを呼び出したり、会話の状態を維持・管理するためのユーティリティをいくつか定義しておきましょう。

In [5]:
@retry(wait=wait_random_exponential(min=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, functions=None, function_call=None, model=GPT_MODEL):
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer " + openai.api_key,
    }
    json_data = {"model": model, "messages": messages}
    if functions is not None:
        json_data.update({"functions": functions})
    if function_call is not None:
        json_data.update({"function_call": function_call})
    try:
        response = requests.post(
            "https://api.openai.com/v1/chat/completions",
            headers=headers,
            json=json_data,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e

In [61]:
def pretty_print_conversation(messages):
    role_to_color = {
        "system": "red",
        "user": "green",
        "assistant": "blue",
        "function": "magenta",
    }
    formatted_messages = []
    for message in messages:
        if message["role"] == "system":
            formatted_messages.append(f"system: {message['content']}\n")
        elif message["role"] == "user":
            formatted_messages.append(f"user: {message['content']}\n")
        elif message["role"] == "assistant" and message.get("function_call"):
            formatted_messages.append(f"assistant: {message['function_call']}\n")
        elif message["role"] == "assistant" and not message.get("function_call"):
            formatted_messages.append(f"assistant: {message['content']}\n")
        elif message["role"] == "function":
            formatted_messages.append(f"function: ({message['name']}): {message['content']}\n")
    for formatted_message in formatted_messages:
        print(
            colored(
                formatted_message,
                role_to_color[messages[formatted_messages.index(formatted_message)]["role"]]
            )
        )


## 基本コンセプト
仮想的な気象APIとのインターフェイスとして、いくつかの関数仕様を作成しましょう。これらの関数仕様をチャット補完APIに渡すことで、仕様に沿った関数引数を生成するようにします。

In [7]:
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "format": {
                    "type": "string",
                    "enum": ["celsius", "fahrenheit"],
                    "description": "The temperature unit to use. Infer this from the users location.",
                },
            },
            "required": ["location", "format"],
        },
    },
    {
        "name": "get_n_day_weather_forecast",
        "description": "Get an N-day weather forecast",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "format": {
                    "type": "string",
                    "enum": ["celsius", "fahrenheit"],
                    "description": "The temperature unit to use. Infer this from the users location.",
                },
                "num_days": {
                    "type": "integer",
                    "description": "The number of days to forecast",
                }
            },
            "required": ["location", "format", "num_days"]
        },
    },
]

現在の天気についてモデルを促すと、明確な質問で応えてくれる。

In [16]:
messages = []
messages.append({"role": "system", "content": "関数に差し込む値について、勝手に決めつけないでください。ユーザーからの要求があいまいな場合は、説明を求めること"})
messages.append({"role": "user", "content": "今日の天気は？"})
chat_response = chat_completion_request(
    messages, functions=functions
)
assistant_message = chat_response.json()["choices"][0]["message"]
messages.append(assistant_message)
assistant_message

{'role': 'assistant',
 'content': 'ごめんなさい、あなたの現在の位置情報が必要です。どの都市の天気を知りたいですか？また、温度は摂氏（celsius）と華氏（fahrenheit）のどちらで表示しますか？'}

不足する情報を提供すると、適切な関数引数を生成してくれます。

（gpt-3.5-turbo-0613を使って日本語の場合、勝手にパラメータを生成してしまうかも）

In [18]:
messages.append({"role": "user", "content": "日本の愛知県にいます。"})
chat_response = chat_completion_request(
    messages, functions=functions
)
assistant_message = chat_response.json()["choices"][0]["message"]
messages.append(assistant_message)
assistant_message

{'role': 'assistant',
 'content': None,
 'function_call': {'name': 'get_current_weather',
  'arguments': '{\n  "location": "愛知県",\n  "format": "celsius"\n}'}}

別の方法で促すことで、伝えた別の機能をターゲットにさせることができるのです。

In [20]:
messages = []
messages.append({"role": "system", "content": "関数に与える引数を勝手に決めつけないでください。ユーザーからの要求があいまいな場合は、説明を求めること。"})
messages.append({"role": "user", "content": "今後n日間の日本の愛知県の天気はどうなるのでしょうか？"})
chat_response = chat_completion_request(
    messages, functions=functions
)
assistant_message = chat_response.json()["choices"][0]["message"]
messages.append(assistant_message)
assistant_message


{'role': 'assistant',
 'content': '確認させて頂きますが、n日間の詳細な予報をお求めですか？また、摂氏(Celsius)か華氏(Fahrenheit)で気温をお求めですか？'}

もう一度言いますが、モデルはまだ十分な情報を持っていないため、我々に説明を求めているのです。この場合、予測する場所はすでに分かっていますが、予測に必要な日数は何日か知る必要があります。

In [22]:
messages.append({"role": "user", "content": "5日間"})
chat_response = chat_completion_request(
    messages, functions=functions
)
chat_response.json()["choices"][0]

{'index': 0,
 'message': {'role': 'assistant',
  'content': None,
  'function_call': {'name': 'get_n_day_weather_forecast',
   'arguments': '{\n  "location": "愛知県",\n  "format": "celsius",\n  "num_days": 5\n}'}},
 'finish_reason': 'function_call'}

## 特定の関数を強制的に使用させる、または関数を使用させない
function_call引数を使うことで、特定の関数、例えばget_n_day_weather_forecastを使うようモデルに強制することができます。そうすることで、その関数の使い方をモデルに強制的に推測させることができます。


In [23]:
# このセルでは、モデルに get_n_day_weather_forecast を使わせています
messages = []
messages.append({"role": "system", "content": "関数に与える引数を勝手に決めつけないでください。ユーザーからの要求があいまいな場合は、説明を求めること。"})
messages.append({"role": "user", "content": "カナダのトロントの天気予報をお願いします。"})
chat_response = chat_completion_request(
    messages, functions=functions, function_call={"name": "get_n_day_weather_forecast"}
)
chat_response.json()["choices"][0]["message"]

{'role': 'assistant',
 'content': None,
 'function_call': {'name': 'get_n_day_weather_forecast',
  'arguments': '{\n  "location": "Toronto, Canada",\n  "format": "celsius",\n  "num_days": 5\n}'}}

In [25]:
# get_n_day_weather_forecast を使うようにモデルに強制しなければ、そうならないかもしれません
messages = []
messages.append({"role": "system", "content": "関数に与える引数を勝手に決めつけないでください。ユーザーからの要求があいまいな場合は、説明を求めること。"})
messages.append({"role": "user", "content": "カナダのトロントの天気予報をお願いします。"})
chat_response = chat_completion_request(
    messages, functions=functions
)
chat_response.json()["choices"][0]["message"]

{'role': 'assistant',
 'content': None,
 'function_call': {'name': 'get_n_day_weather_forecast',
  'arguments': '{\n  "location": "Toronto, Canada",\n  "format": "celsius",\n  "num_days": 5\n}'}}

また、モデルに関数を全く使わせないようにすることもできます。そうすることで、適切な関数呼び出しが行われないようにすることができます。

In [26]:
messages = []
messages.append({"role": "system", "content": "関数に与える引数を勝手に決めつけないでください。ユーザーからの要求があいまいな場合は、説明を求めること。"})
messages.append({"role": "user", "content": "カナダのトロントの現在の天気（摂氏）を教えてください。"})
chat_response = chat_completion_request(
    messages, functions=functions, function_call="none"
)
chat_response.json()["choices"][0]["message"]

{'role': 'assistant',
 'content': 'カナダのトロントの現在の天気（摂氏）を取得するために、天気APIを使用します。しばらくお待ちください。'}

# モデルで生成された引数で関数を呼び出す方法
次の例では、入力がモデルで生成された関数を実行する方法を示し、これを使用してデータベースに関する質問に答えるエージェントを実装します。簡単のために、Chinookサンプルデータベースを使用します。

注意：モデルは正しいSQLを生成することに完璧に信頼できるわけではないので、本番環境ではSQLの生成はハイリスクである可能性があります。

## SQLクエリを実行する関数を指定する
まず、SQLiteデータベースからデータを抽出するための有用なユーティリティ関数を定義しましょう。

In [27]:
import sqlite3

conn = sqlite3.connect("data/Chinook.db")
print("Opened database successfully")

Opened database successfully


In [28]:
def get_table_names(conn):
    """テーブル名のリストを返す"""
    table_names = []
    tables = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
    for table in tables.fetchall():
        table_names.append(table[0])
    return table_names

def get_column_names(conn, table_name):
    """コラム名のリストを返す"""
    column_names = []
    columns = conn.execute(f"PRAGMA table_info('{table_name}');").fetchall()
    for col in columns:
        column_names.append(col[1])
    return column_names

def get_database_info(conn):
    """データベース内の各テーブルのテーブル名とカラムを含むdict型のリストを返す"""
    table_dicts = []
    for table_name in get_table_names(conn):
        column_names = get_column_names(conn, table_name)
        table_dicts.append({"table_name": table_name, "column_names": column_names})
    return table_dicts

これらのユーティリティ関数を使用して、データベーススキーマの表現を抽出することができます。

In [32]:
database_schema_dict = get_database_info(conn)
database_schema_string = "\n".join(
    [
        f"Table: {table['table_name']}\nColumns: {'.'.join(table['column_names'])}"
        for table in database_schema_dict
    ]
)

前回と同様に、APIに引数を生成させたい関数の関数仕様を定義します。データベーススキーマを関数仕様に挿入していることに注目してください。これは、モデルにとって重要なことです。

In [33]:
functions = [
    {
        "name": "ask_database",
        "description": "Use this function to answer user questions about music. Output should be a fully formed SQL query.",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": f"""
                            SQL query extracting info to answer the user's question.
                            SQL should be written using this database schema:
                            {database_schema_string}
                            The query should be returned in plain text, not in JSON.
                            """,
                }
            },
            "required": ["query"],
        },
    }
]

## SQLクエリの実行
では、実際にデータベースに対してクエリを実行する関数を実装してみましょう。

In [58]:
def ask_database(conn, query):
    """指定されたSQLクエリでSQLiteデータベースに問い合わせる関数"""
    try:
        results = str(conn.execute(query).fetchall())
    except Exception as e:
        results = f"query failed with error: {e}"
    return results

def execute_function_call(message):
    if message["function_call"]["name"] == "ask_database":
        query = eval(message["function_call"]["arguments"])["query"]
        results = ask_database(conn, query)
    else:
        results = f"Error: function {message['function_call']['name']} does not exist"
    return results

In [62]:
messages = []
messages.append({"role": "system", "content": "Chinook音楽データベースに対するSQLクエリを生成して、ユーザーの質問に答えてください。"})
messages.append({"role": "user", "content": "こんにちは。トラック数でトップ5のアーティストは誰ですか？"})
chat_response = chat_completion_request(messages, functions)
assistant_message = chat_response.json()["choices"][0]["message"]
messages.append(assistant_message)
if assistant_message.get("function_call"):
    results = execute_function_call(assistant_message)
    messages.append({"role": "function", "name": assistant_message["function_call"]["name"], "content": results})
pretty_print_conversation(messages)

system: Chinook音楽データベースに対するSQLクエリを生成して、ユーザーの質問に答えてください。

user: こんにちは。トラック数でトップ5のアーティストは誰ですか？

assistant: {'name': 'ask_database', 'arguments': '{\n  "query": "SELECT artists.Name, COUNT(tracks.TrackId) as TrackCount FROM artists JOIN albums ON artists.ArtistId = albums.ArtistId JOIN tracks ON albums.AlbumId = tracks.AlbumId GROUP BY artists.ArtistId ORDER BY TrackCount DESC LIMIT 5;"\n}'}

function: (ask_database): [('Iron Maiden', 213), ('U2', 135), ('Led Zeppelin', 114), ('Metallica', 112), ('Deep Purple', 92)]



In [63]:
messages.append({"role": "user", "content": "最も曲数の多いアルバム名はなんですか？"})
chat_response = chat_completion_request(messages, functions)
assistant_message = chat_response.json()["choices"][0]["message"]
messages.append(assistant_message)
if assistant_message.get("function_call"):
    results = execute_function_call(assistant_message)
    messages.append({"role": "function", "name": assistant_message["function_call"]["name"], "content": results})
pretty_print_conversation(messages)

system: Chinook音楽データベースに対するSQLクエリを生成して、ユーザーの質問に答えてください。

user: こんにちは。トラック数でトップ5のアーティストは誰ですか？

assistant: {'name': 'ask_database', 'arguments': '{\n  "query": "SELECT artists.Name, COUNT(tracks.TrackId) as TrackCount FROM artists JOIN albums ON artists.ArtistId = albums.ArtistId JOIN tracks ON albums.AlbumId = tracks.AlbumId GROUP BY artists.ArtistId ORDER BY TrackCount DESC LIMIT 5;"\n}'}

function: (ask_database): [('Iron Maiden', 213), ('U2', 135), ('Led Zeppelin', 114), ('Metallica', 112), ('Deep Purple', 92)]

user: 最も曲数の多いアルバム名はなんですか？

assistant: {'name': 'ask_database', 'arguments': '{\n  "query": "SELECT albums.Title, COUNT(tracks.TrackId) AS TrackCount FROM albums JOIN tracks ON albums.AlbumId = tracks.AlbumId GROUP BY albums.AlbumId ORDER BY TrackCount DESC LIMIT 1;"\n}'}

function: (ask_database): [('Greatest Hits', 57)]

